# Genus of Traces
This code uses properties of Markoff equations to POSSIBLY narrow down the possible genus and then uses the algorithm of Goldstein and Turner to calculate the genus of individual matrices

In [1]:
import numpy as np
import pandas as pd
import ast
import matplotlib.pyplot as plt
from itertools import groupby, permutations

Import in your csv that was cleaned up already

In [2]:
#Again which 2 lines are commented and which 2 are commented out depend on your preference
#df=pd.read_csv("../CSV/upto100000/admissible_traces_evenbigger.csv")
#df_traces=pd.read_csv("../CSV/upto100000/admissible_trace_only_evenbigger.csv")
df=pd.read_csv("../CSV/admissible_traces_converted.csv")
df_traces=pd.read_csv("../CSV/admissible_trace_only_converted.csv")
df['matrix']=[ast.literal_eval(m) for m in df.matrix.values]
df['decomp']=[ast.literal_eval(d) for d in df.decomp.values]

#only take the values that are in the commutator subgroup
newRows=[]
for t in df_traces.trace.values:
    #grab all of the rows in the dataframe with the same trace and is in the commutator subgroup
    list_to_be_sorted=df.loc[(df.trace==t) & (df.commutator==1)].values.tolist()
    first_portion=[]
    matrix_reps=[]
    # we consider transposes to be duplicates and so we remove them from the search
    for m in list_to_be_sorted:
        minipulate=[[m[2][0][0],m[2][1][0]],[m[2][0][1],m[2][1][1]]]
        if minipulate not in matrix_reps: 
            matrix_reps.append(m[2])
            first_portion.append(m)

    newRows=newRows+first_portion
commutator_elements_only=pd.DataFrame(newRows,columns=['trace','admissible','matrix','decomp','length','final_position','commutator'], )

# Markoff Portion inspired by Ghosh and Sarnak

In [3]:
# If t>0 then this is how we find the possible triples Ghosh and Sarnak
def posMark(t):
    k=t+2
    alltrips=[]
    bound=int(round(np.sqrt(abs(k)))+2)
    for u3 in range(6,bound,4):
        for u2 in range(6, u3+1, 4):
            temp=(u2*u3)**2-4*(u2**2+u3**2-k)
            if temp>0:
                u1=int(round((-u3*u2+np.sqrt(temp))/2))
                if u1**2+u2**2+u3**2+u1*u2*u3==k and u1 %4 ==2 and u1<=u2:
                    alltrips.append((-u1, u2, u3))
    return alltrips

# If t<0 then this is how we find the possible triples Ghosh and Sarnak
def negMark(t):
    k=t+2
    alltrips=[]
    bound=int(round(np.sqrt(abs(k)))+2)
    for u1 in range(6, bound,4):
        for u2 in range(u1, bound,4):
            temp=(u2*u1)**2-4*(u2**2+u1**2-k)
            if temp>=0:
                u3=int(round((u1*u2+np.sqrt(temp))/2))
                u3c=int(round((u1*u2-np.sqrt(temp))/2))
                if u1**2+u2**2+u3**2-u1*u2*u3==k:
                    if u3<=int(round(u1*u2/2)) and u2<=u3 and u3%4==2:
                            alltrips.append((u1, u2, u3))
                    elif temp!=0 and u3c<=int(round(u1*u2/2)) and u2<=u3c and u3c%4==2:
                            alltrips.append((u1, u2, u3c))
    return alltrips


#checks to see which version of the triples we are looking for
def Markoff(t):
    if t>0:
        return posMark(t)
    else:
        return negMark(t)

If there exists no Markoff Triples for t, then it has to be at least a 2-commutator so return False. If we get to this point, then we need to see if any of the triples give us a solution these triples are saved as unknowns and can be searched in the mathematic file Markoff.nb to see if it is a 1-commutator.  If you first run the search to bound above (FunctionA), then you know that if the Mathematica file says there are no possibilities, then it is a 2-commutator

In [4]:
def findingTriples(t):
    listofmark=Markoff(t)
    if len(listofmark)==0:
        return False
    return listofmark     

# Goldstein and Turner Portion

In [5]:
#takes a matrix and converts it to a's b's c's and d's
def convertToAandB(matrix):
    asandbs=[]
    for i in range(0,len(matrix[0])):
        if matrix[0][i]>0:
            asandbs=asandbs+['a']*matrix[0][i]
        elif matrix[0][i]<0:
            asandbs=asandbs+['c']*(-matrix[0][i])
        if matrix[1][i]>0:
            asandbs=asandbs+['b']*matrix[1][i]
        elif matrix[1][i]<0:
            asandbs=asandbs+['d']*(-matrix[1][i])

    return asandbs  

In [6]:
def computeRankMod2(size, matrix):
	rowsWithPiviot = []
	#for each column
	for i in range(size):
		#pivot row
		pivotRow=None
		piv=None
		#for each rows
		for j in range(size):
			# if it is not already a pivot
			if j not in rowsWithPiviot:
				#position in vector version of matrix
				pos=size*j+i
				#looking for a value of 1 in the pivot (since mod 2)
				if matrix[pos]==1:
					#a row we care about i.e. either it is going to be our pivot
					if pivotRow==None:
						pivotRow = j
						piv = matrix[pos:size*(j+1)]
						rowsWithPiviot.append(j)
					# or we actually need to subtract pivot from this row
					else:
						for k in range(size):
							if k not in rowsWithPiviot:
								startPos = size*k+i
								#only subtract pivot row from it if it starts with a 1
								if matrix[startPos]==1:
									for m in range(size-i):
										matrix[startPos+m]=(matrix[startPos+m]-piv[m])%2
	return len(rowsWithPiviot)

In [7]:
# Finds the rank of the matrix cooresponding to the graph
def makeTheMatrix(thePositions, size):
    mini_matrix=[0]*(size**2)
    finalPositions = [-1]*size
    for i in range(size):
        prod = i*size
        start = finalPositions[i]+1
        if start==0:
            start = 2*i+1
        for j in range(start,2*size):
            theIndex= thePositions[j]-1
            if theIndex==i:
                break
            mini_matrix[prod+theIndex]^=1
            if finalPositions[theIndex]==-1:
                finalPositions[theIndex]=j
    return int(computeRankMod2(size, mini_matrix)/2)

In [8]:
#  function that finds the line segments for each point.
# It will also graph the circle if plot_it is set to True
def makeTheGraph(pair,plot_it,size):
    #create the evenly space points on the circle
    thelines=[]
    ThetaArray = np.linspace(0, 2*np.pi, len(pair), endpoint=False)
    x = np.cos(ThetaArray)
    y = np.sin(ThetaArray)
    
    #find the endpoints of the linesegments intersecting the points from pair
    for i in range(0,size):
        pt1=pair.index(i+1)
        pt2=pair.index(i+1,pt1+1,len(pair))
        thelines.append([x[pt1],y[pt1],x[pt2],y[pt2]])
        
    #graph it if you want
    if plot_it:
        plt.figure(figsize=(8.0, 8.0))

        # also set the x and y axis limits
        plt.xlim(-1.2, 1.2)
        plt.ylim(-1.2, 1.2)

        # plot the x,y points, connecting successive points with lines
        plt.scatter(x,y,label=pair)
        for i in range(0,size):
            pt1=pair.index(i+1)
            pt2=pair.index(i+1,pt1+1,len(pair))
            plt.plot([x[pt1],x[pt2]],[y[pt1],y[pt2]])
        
        plt.plot()
        print(plt.show())
    #returns the "lines"
    return thelines

This next function takes our list of 'a', 'b', 'c', 'd' and creates all possible matchings between the 'a's and 'c's and the 'b''s and 'd''s, saves the genus if it ends up being the best so far

In [9]:
def matching_pairs(asbs, sizes,smallest, graph):
    best_pairing=0;
    best_genus= int(2*np.floor(sizes))
    mini=[[i,asbs[i]] for i in range(0,len(asbs))]
    thea=[ x for x in mini if x[1]=='a']
    theb=[ x for x in mini if x[1]=='b']
    
    #possible combos of d's and c's
    thec=list(permutations([ x for x in mini if x[1]=='c']))
    thed=list(permutations([ x for x in mini if x[1]=='d']))
    
    for cval in thec:
        for dval in thed:
            #for each chosen permuation, assign the numbers to describe which points are connected by an arc
            found=1;
            thispos=[0]*len(mini)
            for i in range(0,len(mini)):
                    if thispos[i]==0:
                        if mini[i][1]=='a':
                            thispos[i]=found
                            thispos[mini.index(cval[thea.index([i,'a'])])]=found
                            found+=1
                        elif mini[i][1]=='b':
                            thispos[i]=found
                            thispos[mini.index(dval[theb.index([i,'b'])])]=found
                            found+=1
                        elif mini[i][1]=='c':
                            thispos[i]=found
                            thispos[mini.index(thea[cval.index([i,'c'])])]=found
                            found+=1
                        else: 
                            thispos[i]=found
                            thispos[mini.index(theb[dval.index([i,'d'])])]=found
                            found+=1
            minigenus=makeTheMatrix(thispos,sizes)
            if minigenus<best_genus:
                best_genus=minigenus
                best_pairing=thispos
                #print(best_genus)
                #ayo we are done
                if best_genus==smallest:
                    makeTheGraph(best_pairing,graph,sizes)
                    return [best_genus, best_pairing]

    makeTheGraph(best_pairing,graph,sizes)
    return [best_genus, best_pairing ]
                            

In [10]:
def find_the_genus(matrix_rep, smallest, graph):
    sizes=int(sum(np.absolute(matrix_rep[0])+np.absolute(matrix_rep[1]))/2)
    allPossibleMatches=matching_pairs(convertToAandB(matrix_rep), sizes,smallest, graph)
    return allPossibleMatches


## Function to find the genus of a single matrix
def find_the_genusA(matrix_rep, graph):
    return find_the_genus(matrix_rep, 1, graph)

# FunctionA 
i.e. Check that the commutator-genus of the trace is bounded by 2

In [11]:
## looping through and running it everywhere
#Commutator constructors really just checks that its bounded by 2
upperbound=[0]*len(df_traces.trace.values)
counter=0
for t in df_traces.trace.values:
    
    gonna_loop_through=commutator_elements_only.loc[commutator_elements_only.trace==t].decomp.values
    genuses=[0]*len(gonna_loop_through)
    
    for big_loop in range(len(gonna_loop_through)):
        matrix_rep=gonna_loop_through[big_loop]
        
        #if the length is <=3 print, you are done
        if len(matrix_rep[0])<=3: 
            upperbound[counter]=1
            break
        else:
            #for now i don't care about the best pairing just what its genus is
            genuses[big_loop]=find_the_genusA(matrix_rep, False)[0]
            if genuses[big_loop]<=2:
                upperbound[counter]= genuses[big_loop]
                break
    #finding the samallest trace if we didn't know it was already 1 from prior           
    if upperbound[counter]==0:
        if len(genuses)>0:
            upperbound[counter]=min([i for i in genuses if i>0])
            print("nope!",[t,upperbound[counter]])
        else:
            print(t," there are no commutator subgroups here to begin with")
    if counter%10==0:
        print(t)
    counter=counter+1
    

18
578
1006  there are no commutator subgroups here to begin with
1262
1726  there are no commutator subgroups here to begin with
1938
2322
2558  there are no commutator subgroups here to begin with
2882
3566
4242
4626
4718  there are no commutator subgroups here to begin with
5186
5870
6546
6930
7490
8174
8850
9234
9794
10478
11154
11538
12098
12782
13458
13842
14402
15086
15762
16146
16706
17390
18066
18450
19010
19694
20370
20754
21314
21998
22674
23058
23618
24302
24978


# FunctionB 
Getting Commutator-Genus of traces unless there exists a markoff triple to check and the reps are getting too big

In [12]:
#smallest genus will contain the actual genus if it could be proven and if not, it will 
#hold markoff triples and the matrices it was not able to search due to length
smallestGenus=[0]*len(df_traces.trace.values)
counter=0
for t2 in df_traces[df_traces.trace<40000].trace.values:
    t=t2
    smallestpossible=1
    
    #getting the correct sign associated with t for Markoff since that is the only place it matters
    if t2% 256 not in [2,18,66,146]:
        t=-t2
    
    #checking to see if it is of an exceptional form t=16a^2+2
    if t>0 and 16*(int(np.sqrt((t-2)/16))**2)+2==t:
        smallestGenus[counter]=1
    else:
        #take into account the fact that if there are no markoff triples of the form then clearly no 1-commutator exists
        all_triples=findingTriples(t)
        if all_triples==False:
            smallestpossible=2
        
        #loop through all possibilities
        gonna_loop_through=commutator_elements_only.loc[commutator_elements_only.trace==t2]
        genuses=[0]*len(gonna_loop_through)
    
        for big_loop in range(len(gonna_loop_through)):
            matrix_rep=gonna_loop_through.decomp.values[big_loop]
    
            
            #if the length is <=3 print, you are done
            if len(matrix_rep[0])<=3: 
                smallestGenus[counter]=1
                break
            #too big leave it for mathematica and we have some result so far
            elif gonna_loop_through.length.values[big_loop]>20 and len([i for i in genuses if i>0])>0:
                genuses[big_loop]=-1
                break
            else:
                #for now i don't care about the best pairing just what its genus is
                genuses[big_loop]=find_the_genus(matrix_rep,smallestpossible, False)[0]

                if genuses[big_loop]==smallestpossible:
                    break
        #either its clearly a 1-commutator, it is a 2-commutator, its larger than a 2-commutator, 
        # or we are going to use Markoff to differentiate between 1 and 2 commutators   
        if smallestGenus[counter]==0:
            if len([i for i in genuses if i>0])==0:
                print(t," there are no commutator subgroups here to begin with")
            elif min([i for i in genuses if i>0])==1:
                smallestGenus[counter]=1
            elif  min([i for i in genuses if i>0])==2:
                if smallestpossible==2:
                    smallestGenus[counter]=2
                elif min([i for i in genuses])==-1:
                    #grab the matrices we didn't check and put them in instead
                    thematrices=gonna_loop_through.loc[gonna_loop_through.length>20].matrix.values
                    smallestGenus[counter]=[all_triples, thematrices]
                else:
                    smallestGenus[counter]=2
            else:
                smallestGenus[counter]=min([i for i in genuses if i>0])
                print("Weird, we usually get a 2-commutator by now, I reccomend doing upperbound first with this value", t2, smallestGenus[counter])
    
    #just a check to see where we are currently
    if counter%10==0:
        print(t2)
    counter=counter+1

18
578
-1006  there are no commutator subgroups here to begin with
1262
-1726  there are no commutator subgroups here to begin with
1938
2322
-2558  there are no commutator subgroups here to begin with
2882
3566
4242
4626
-4718  there are no commutator subgroups here to begin with
5186
5870
6546
6930
7490
8174
8850
9234
9794
10478
11154
11538
12098
12782
13458
13842
14402
15086
15762
16146
16706
17390
18066
18450
19010
19694
20370
20754
21314
21998
22674
23058
23618
24302
24978


Save the results as a csv

In [13]:
exporting_dataframe=pd.DataFrame({"trace":df_traces.trace,"L_class_number": df_traces.L_class_number, "commutator_class_number":df_traces.commutator_class_number, "genus": smallestGenus})
#exporting_dataframe.to_csv("../CSV/genus_info.csv", index=False)
#exporting_dataframe.to_csv("../CSV/upto100000/genus_info100000.csv", index=False)

In [14]:
exporting_dataframe

,trace,L_class_number,commutator_class_number,genus
0,18,1,1,1
1,66,2,1,1
2,110,4,2,1
3,146,3,1,1
4,254,12,2,1
...,...,...,...,...
426,24642,512,72,2
427,24766,400,10,"[[(6, 70, 90)], [[[-23447, -1488], [-20784, -1..."
428,24834,496,60,1
429,24942,152,60,1
